In [148]:
import pickle
import numpy as np
import tensorflow as tf
import keras

import os
import datetime

from keras.layers import Dense
from Chess import construct_board
from Chess.view import view_board_colour
from sklearn.model_selection import train_test_split

from ipywidgets import interact
import ipywidgets as widgets

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

In [128]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Open our set of randomly selected Board objects
The script `generate_data.py` pickles a list of board objects after a sequence of moves is run through. This leaves us with a long list of `Board` objects which we can now use!

In [2]:
with open("generated_data/boards.pickle", 'rb') as f:
    boards = pickle.load(f)

board_list = boards

In [3]:
view_board_colour(board_list[0])
view_board_colour(board_list[1])

Black to move - turn 8
 R  N        K  B     R 
 P  B  P  P     P     P 
    P        P          
                   Q  P 
          P  P     B    
 P     N        N       
    P  P        P  P  P 
 R        Q  K        R 

Black to move - turn 13
 R        Q             
 P  P  P     K  P  P  Q 
       N  B  P  N  P    
                   B    
          P  N          
          B             
 P  P  P     K  P  P    
 R                      



## Data processing
Now the data needs to be formed into a reasonable type for training. This section will get all the positions which can be moved to as a list for each game, and all the positions of the pieces for each board as training data. Initally, this won't account for the piece type, which side is to move, or the different types of pieces and moves.

In [4]:
# Using set comprehension to get only the unique elements of each boards moves (the target data)
valid_move_list = [list({i for i in board.allied_moves.all_valid}) for board in board_list]

# We are targeting the correct prediction of each of these squares as "active"
len(valid_move_list[0])

26

In [5]:
# Get all the piece types and positions from the location map of each board (training data)
piece_position_list = [{position: piece.kind for position, piece in board.loc_map.items()} for board in board_list]

# This will be at most 32 pieces
len(piece_position_list[0])

30

In [6]:
# Encode the location of any piece as a 1, and all other positions as 0
feature = []
for instance in piece_position_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    feature.append(array.flatten())

In [7]:
# Simple encoding will be used at first, where any piece present is represented by a one and not present by zero.
target = []
for instance in valid_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    target.append(array.flatten())
    
print(target[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.]


In [159]:
X = np.array(feature)
Y = np.array(target)

print(X.shape)
print(Y.shape)

(1000, 64)
(1000, 64)


The data needs to be split into training and testing portions using the `sklearn` modules implementation.

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

Beginning with a simple sequential model, which takes in a 1x64 array of features and outputs a prediction at each of the 64 squares.

In [144]:
# Inspired by the model at https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

model = keras.Sequential()
model.add(Dense(128, input_dim=64, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='sigmoid'))

In [145]:
model.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy", "binary_accuracy"]
)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_18 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_19 (Dense)             (None, 64)                16448     
Total params: 57,792
Trainable params: 57,792
Non-trainable params: 0
_________________________________________________________________


In [149]:
# From https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-12-01 03:02:48.128969: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-01 03:02:48.129010: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-01 03:02:48.129217: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [150]:
model.fit(
    X_train, 
    Y_train, 
    epochs=100, 
    batch_size=20,
    callbacks=[tensorboard_callback]
)

Epoch 1/100
26/40 [==================>...........] - ETA: 0s - loss: 0.3631 - accuracy: 0.0308 - binary_accuracy: 0.8392

2021-12-01 03:02:54.366161: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-01 03:02:54.366173: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-01 03:02:54.373342: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-01 03:02:54.373604: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-01 03:02:54.374082: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20211201-030248/train/plugins/profile/2021_12_01_03_02_54
2021-12-01 03:02:54.374371: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20211201-030248/train/plugins/profile/2021_12_01_03_02_54/Elliss-MacBook-Air.local.trace.json.gz
2021-12-01 03:02:54.374573: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20211201-030248/train/plugins

40/40 [==============================] - 0s 6ms/step - loss: 0.3635 - accuracy: 0.0325 - binary_accuracy: 0.8394
Epoch 2/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3553 - accuracy: 0.0425 - binary_accuracy: 0.8463
Epoch 3/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3491 - accuracy: 0.0375 - binary_accuracy: 0.8488
Epoch 4/100
40/40 [==============================] - 0s 6ms/step - loss: 0.3402 - accuracy: 0.0412 - binary_accuracy: 0.8538
Epoch 5/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3378 - accuracy: 0.0487 - binary_accuracy: 0.8572
Epoch 6/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3288 - accuracy: 0.0525 - binary_accuracy: 0.8612
Epoch 7/100
40/40 [==============================] - 0s 6ms/step - loss: 0.3214 - accuracy: 0.0475 - binary_accuracy: 0.8669
Epoch 8/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3162 - accuracy: 0.0437 - binary_accuracy: 0.8706
Epoch 9/100


40/40 [==============================] - 0s 5ms/step - loss: 0.0980 - accuracy: 0.0850 - binary_accuracy: 0.9771
Epoch 67/100
40/40 [==============================] - 0s 5ms/step - loss: 0.0960 - accuracy: 0.0850 - binary_accuracy: 0.9784
Epoch 68/100
40/40 [==============================] - 0s 5ms/step - loss: 0.0931 - accuracy: 0.0875 - binary_accuracy: 0.9796
Epoch 69/100
40/40 [==============================] - 0s 6ms/step - loss: 0.0905 - accuracy: 0.0912 - binary_accuracy: 0.9812
Epoch 70/100
40/40 [==============================] - 0s 5ms/step - loss: 0.0881 - accuracy: 0.0912 - binary_accuracy: 0.9816
Epoch 71/100
40/40 [==============================] - 0s 5ms/step - loss: 0.0855 - accuracy: 0.0837 - binary_accuracy: 0.9828
Epoch 72/100
40/40 [==============================] - 0s 5ms/step - loss: 0.0837 - accuracy: 0.0825 - binary_accuracy: 0.9836
Epoch 73/100
40/40 [==============================] - 0s 6ms/step - loss: 0.0817 - accuracy: 0.0850 - binary_accuracy: 0.9842
Epoch

In [151]:
loss, accuracy, binary_accuracy = model.evaluate(X_test, Y_test)

7/7 [==============================] - 0s 8ms/step - loss: 1.4705 - accuracy: 0.0350 - binary_accuracy: 0.6895


2021-12-01 03:03:23.846142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


## Visualising the output
Now both the raw output of the model and the binarised version can be displayed interactively.

In [136]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 80534), started 0:02:51 ago. (Use '!kill 80534' to kill it.)

In [152]:
def predict_val(val):
    prediction = model.predict(val.reshape(1, 64))
    prediction = prediction.reshape(8, 8)
    return np.where(prediction > 0.5, 1, 0)

def predict_raw(val):
    prediction = model.predict(val.reshape(1, 64))
    return prediction.reshape(8, 8)    

In [153]:
def find_diff(n):
    Y_instance = Y_test[n].reshape(8, 8)
    prediction = predict_val(X_test[n])
    
    diff = np.where(prediction == Y_instance, 1, 0)

    return diff

In [154]:
sns.set_theme(style="white")
sns.color_palette("crest", as_cmap=True)
sns.set_palette("crest")

def plot_sns(val):
    # From https://seaborn.pydata.org/examples/many_pairwise_correlations.html
    sns.heatmap(val)
    plt.show()

def plot(val):
    cmap = colors.ListedColormap(['white', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val, cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));

#     sns.heatmap(val)
    plt.show()

def plot_diff(val):
    cmap = colors.ListedColormap(['red', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val, cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));
    
#     sns.heatmap(val)
    plt.show()

In [155]:
def generate_plots(
            n, target=True, 
            prediction=True, 
            raw_prediction=True, 
            diff=True
        ):
    if target:         plot(Y_test[n].reshape(8, 8))    
    if prediction:     plot(predict_val(X_test[n]))
    if raw_prediction: plot_sns(predict_raw(X_test[n]))
    if diff:           plot_diff(find_diff(n))

In [156]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
interact(
    generate_plots, 
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, target=True, prediction=True, raw_prediction=True, diff=True)>